In [28]:
import os

[file.replace('.csv','') for file in filter(lambda x:x.endswith('.csv'), os.listdir('nasdaq'))]

['AGEN',
 'BTU',
 'TSHA',
 'ALZN',
 'CLSD',
 'XERS',
 'CCL',
 'CNTX',
 'CAPL',
 'SCNI',
 'SONM',
 'BDSX',
 'ANVS',
 'PRLD',
 'DADA',
 'BVS',
 'ASAN',
 'MYPS',
 'ALTO',
 'AVIR',
 'CFLT',
 'ALKT',
 'CRCT',
 'CPA',
 'WRN',
 'GOEV',
 'SQNS',
 'MGNX',
 'CGNX',
 'ATRA',
 'RCON',
 'GBIO',
 'BEAT',
 'GSIT',
 'RDI',
 'RGLS',
 'ZNTL',
 'VSTM',
 'CHRW',
 'SABS',
 'ALEC',
 'MIST',
 'CSAN',
 'COGT',
 'CCJ',
 'AKYA',
 'TAIT',
 'CASY',
 'ACLS',
 'STRR',
 'SST',
 'CSTE',
 'XGN',
 'VNET',
 'CNMD',
 'CVGI',
 'SINT',
 'BORR',
 'ARIS',
 'SNOA',
 'ATRC',
 'CLLS',
 'CABA',
 'BBU',
 'CRWD',
 'AY',
 'CGAU',
 'EVO',
 'AMSC',
 'ASC',
 'CIVI',
 'BPMC',
 'BLKB',
 'ORGN',
 'BLDR',
 'CDRE',
 'AM',
 'EMKR',
 'NNBR',
 'AQN',
 'A',
 'CAN',
 'BNGO',
 'COLL',
 'PRT',
 'NRBO',
 'CMSC',
 'NRXP',
 'CSTM',
 'BEAM',
 'ADAP',
 'MCRB',
 'EVOK',
 'PLUG',
 'TKNO',
 'BE',
 'AR',
 'CNEY',
 'CTKB',
 'KNDI',
 'BTAI',
 'RMTI',
 'IDN',
 'ACAD',
 'ALGT',
 'ANGO',
 'YGMZ',
 'AMAT',
 'NINE',
 'TRVI',
 'IRIX',
 'MYO',
 'CGTX',
 'AXSM',
 '

In [32]:
data={'a':[55]}
data.get('a').append(66)
data


{'a': [55, 66]}

In [33]:
import os

data = {}
for file in list(filter(lambda x:x.endswith('.csv'), os.listdir('nasdaq')))[:10]:
        iter = pd.read_csv(f"nasdaq/{file}").iterrows()
        while True:
            try:
                i,row=next(iter)
                d = data.get(row['date'])
                entry = {'ticker': file.replace('.csv',''), 'open': row['open'], 'high': row['high'], 'low': row['low'], 'close': row['close'], 'volume': row['volume']}
                if d==None:
                    data[row['date']] = [entry]
                else:
                    data[row['date']].append(entry)
            except StopIteration:
                break
len(list(data.keys()))

503

In [43]:
mQTY=20000
min_volume=200000
amt=5000
def buy(amt, entry):
    o=entry['open']
    qty=min(mQTY, 100*int(amt/o/100))
    return (entry['ticker'], o,qty,amt-qty*o,o)

print(len(data.get('2024-03-20')))
selected_data = data
positions=[buy(amt, entry) for entry in selected_data]
positions

10


TypeError: string indices must be integers, not 'str'

In [ ]:
import pandas as pd

mQTY=20000
min_volume=200000

def split_list(lst, n):
    # Calculate chunk size
    k, m = divmod(len(lst), n)
    return [lst[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n)]

def buy(ticker, amt, row):
    o=row['open']
    qty=min(mQTY, 100*int(amt/o/100))
    return (ticker, o,qty,amt-qty*o,o)

debug=True

def start(data, amt, positions, ticker=0):
    tickers = [ticker for (ticker, iter) in iters]
    positions=[buy(ticker, amt, next(iter)[1]) for (ticker, iter) in iters]
    name,o,qty,balance,lasto=positions[ticker]
    
    #print("positions",positions)
    count=0
    pamt=qty*o+balance
    while ndays>0:
        try:
            ndays -= 1
            count += 1
            done=False
            index,crow=next(iters[ticker][1])
            d = crow['date']
            name,o,qty,cbalance,lasto = positions[ticker]
            positions[ticker]=(name,o,qty,cbalance,crow['open'])
            amt=qty*crow['open']+cbalance
            delta=0
            s=()
            #print("+++",count,tickers[ticker],crow['open'], f"{int(10000*((crow['open']-o)/o))/100}%",qty,cbalance,amt,delta)
            for i, iter in enumerate(iters):
                if (i!=ticker):
                    index,row=next(iter[1])
                    if d!=row['date']: raise Exception(f'invalid date d:{d} - {tickers[ticker]} - {tickers[i]} - {row['date']}')
                    name, o,qty,balance,lasto = buy(tickers[i], amt, row)
                    #print(f"{tickers[i]} qty:{qty} pqty:{positions[i][2]} o:{o} po:{positions[i][1]} delta:{delta}")
                    if row['volume']>min_volume & qty>positions[i][2] and qty-positions[i][2]>delta:
                        delta=qty-positions[i][2]
                        s=(i,(name, o,qty,balance,row['open']))
            if (s!=()):
                name1, o1,qty1,balance1,lasto1 = positions[ticker]
                name2, o2,qty2,balance2,lasto2 = s[1]
                amt2=qty2*o2+balance2
                if debug: print(f"{d}: {qty1}x{name1}@{crow['open']}+{cbalance} => {qty2}x{name2}@{o2}+{balance2}/{amt2}")
                ticker=s[0]
                positions[ticker]=s[1]
                pamt=amt2
        except StopIteration:
            break
    return positions[ticker]

total_amt=0
amt = 25000
debug=True
ngroups=4
random.shuffle(nasdaq_tickers)
selected_data = data
for tickers in split_list(nasdaq_tickers, ngroups):
    positions=[buy(ticker, amt, next(iter)[1]) for ticker in selected_data]
    ticker,o,qty,balance,lasto=start(selected_data, amt, positions)
    total_amt += qty*lasto+balance
    print(f"*** size:{len(tickers)} {qty}x{ticker}@{lasto} balance:{balance} value:{qty*lasto+balance} gain:{int(10000*(lasto-o)/o)/100}%")
print(ngroups,"invest",ngroups*amt,"gain",total_amt-ngroups*amt, 
      f"{int(10000*(total_amt-ngroups*amt)/(ngroups*amt))/100}%")